<a href="https://colab.research.google.com/github/janithcyapa/Decentralized-HVAC-Control-System/blob/main/energy-plus-utility.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Usage of Energy Plus Utility**

## **Installation**

In [5]:
!pip install -q "energy-plus-utility @ git+https://github.com/mugalan/energy-plus-utility.git@dev"

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [6]:
from eplus import prepare_colab_eplus
prepare_colab_eplus()  # raises on failure, otherwise silent
# TODO: Allow user to specify EPLUS_DIR when running the prepare_colab_eplus()

In [16]:
from eplus import EPlusUtil, EPlusSqlExplorer
import subprocess, json, pathlib, os
import pandas as pd
import json

## **Initialization**

### **Selection of IDF files from examples**

#### **Initial implementation**
`1ZoneDataCenterCRAC_wApproachTemp.idf`
- Single-zone data center
- CRAC unit (Computer Room Air Conditioner)
- Simple, stable HVAC topology
- Real HVAC system
- Clear air mass flow path
- Clear supply air temperature control

Or use
`1ZoneUncontrolled.idf + ZoneHVAC:IdealLoadsAirSystem`
- Fast, conceptual, controller-logic sandbox
- Single thermal zone
- Simple envelope & loads
- Infinite heating/cooling capacity
- Zone temperature/humidity control


#### **Decentralized Controlling**
`5ZoneAirCooledWithDOASAirLoop.idf`.
- DOAS (Dedicated Outdoor Air System)
- Separation of ventilation & conditioning
- Multi-zone building
- Central AHU
- Outdoor air handling
- Zone-level terminals
- Mass flow modulation
- Ventilation control logic

Implementation
- Decentralized zone controller - VAV / terminal unit control
- Zone mass flow rate command - AirTerminal:SingleDuct:*
- Central AHU coordinator - AirLoopHVAC
- AHU supply air temperature - Setpoint Manager (SAT)
- AHU humidity ratio - Humidity control on air loop
- CO₂ coordination - Controller:MechanicalVentilation
- Aggregate ventilation logic - Outdoor Air Controller

As a Secondary / fallback try `5ZoneAirCooled.idf`
- Multiple zones
- Central air system
- Coils, fans, schedules

In [41]:
# Specify the EnergyPlus Import
EPLUS_DIR = "/root/EnergyPlus"
OUT_DIR = "/content/eplus_out"

# Define the Simulation Model
IDF = f"{EPLUS_DIR}/ExampleFiles/5ZoneAirCooled.idf"
# IDF = f"{EPLUS_DIR}/ExampleFiles/2ZoneDataCenterHVAC_wEconomizer.idf"
# Select Weather Data
EPW = f"{EPLUS_DIR}/WeatherData/USA_CA_San.Francisco.Intl.AP.724940_TMY3.epw"


In [42]:
# Create utility instance and set simulation model
util = EPlusUtil(verbose=1, out_dir = OUT_DIR)
# Reset and set model
util.delete_out_dir()
util.set_model(idf = IDF, epw = EPW,outdoor_co2_ppm=400.0, per_person_m3ps_per_W=3.82e-08)
util.ensure_output_sqlite()
util.enable_runtime_logging()

### **Visualisation of IDF files**

In [43]:
# Convert the model to .json
idf_path = pathlib.Path(IDF)
converter = os.path.join(EPLUS_DIR, "ConvertInputFormat")   # on Windows it's ConvertInputFormat.exe
subprocess.run([converter, str(idf_path)], check=True)
epjson_path = idf_path.with_suffix(".epJSON")

In [11]:
# @title
def list_epjson_tables(epjson: dict):
    rows = []
    for obj_type, objs in epjson.items():
        count = len(objs) if isinstance(objs, dict) else "-"
        rows.append((obj_type, count))

    print(f"{'Object Type':45} | {'# Objects'}")
    print("-" * 60)
    for r in sorted(rows):
        print(f"{r[0]:45} | {r[1]}")

def show_object_table(epjson: dict, object_type: str):
    table = epjson.get(object_type)
    if table is None:
        print(f"{object_type} not found")
        return

    print(f"{object_type}")
    print("-" * len(object_type))
    for name, fields in table.items():
        print(f"\n{name}")
        for k, v in fields.items():
            print(f"  {k:35} : {v}")
def find_mass_flow_objects(epjson: dict):
    keywords = ["air_flow", "mass_flow", "flow_rate"]
    rows = []

    for obj_type, objs in epjson.items():
        if not isinstance(objs, dict):
            continue
        for name, fields in objs.items():
            for f in fields:
                if any(k in f.lower() for k in keywords):
                    rows.append((obj_type, name, f, fields[f]))

    print(f"{'Object Type':35} | {'Name':25} | {'Field':30} | Value")
    print("-" * 110)
    for r in rows:
        print(f"{r[0]:35} | {r[1]:25} | {r[2]:30} | {r[3]}")

In [44]:
with open(str(epjson_path), "r", encoding="utf-8") as f:
    epjson = json.load(f)

# list_epjson_tables(epjson)
# show_object_table(epjson, "AirLoopHVAC")
find_mass_flow_objects(epjson)

Object Type                         | Name                      | Field                          | Value
--------------------------------------------------------------------------------------------------------------
AirLoopHVAC                         | VAV Sys 1                 | design_supply_air_flow_rate    | Autosize
AirTerminal:SingleDuct:VAV:Reheat   | SPACE1-1 VAV Reheat       | constant_minimum_air_flow_fraction | 0.3
AirTerminal:SingleDuct:VAV:Reheat   | SPACE1-1 VAV Reheat       | maximum_air_flow_rate          | Autosize
AirTerminal:SingleDuct:VAV:Reheat   | SPACE1-1 VAV Reheat       | maximum_hot_water_or_steam_flow_rate | Autosize
AirTerminal:SingleDuct:VAV:Reheat   | SPACE1-1 VAV Reheat       | minimum_hot_water_or_steam_flow_rate | 0.0
AirTerminal:SingleDuct:VAV:Reheat   | SPACE1-1 VAV Reheat       | zone_minimum_air_flow_input_method | Constant
AirTerminal:SingleDuct:VAV:Reheat   | SPACE2-1 VAV Reheat       | constant_minimum_air_flow_fraction | 0.3
AirTerminal:SingleD

## **Simulations**

Dry run to create tables etc.

In [45]:
util.dry_run_min(include_ems_edd=False)

EnergyPlus Starting
EnergyPlus, Version 25.1.0-68a4a7c774, YMD=2025.12.17 17:59
Initializing Response Factors
Calculating CTFs for "ROOF-1"
Calculating CTFs for "WALL-1"
Calculating CTFs for "FLOOR-SLAB-1"
Calculating CTFs for "INT-WALL-1"
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone and Enclosure Report Variables
Initializing Surface (Shading) Report Variables
Computing Interior Solar Absorption Factors
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializing Solar Calculations
Initializing Surfaces
Initializing Outdoor environment for Surfaces
Setting up Surface Reporting Variables
Initializing Temperature and Flux Histories
Initializing Window Shading
Computing Interior Absorption Factors
Computing Interior Diffuse Solar Absorption Factors
Initializing Solar Heat Gains
Initializing Internal Heat Gains
Initializing Interior Solar Distribution
Initializing Interior

0

In [46]:
util.list_zone_names()

['PLENUM-1', 'SPACE1-1', 'SPACE2-1', 'SPACE3-1', 'SPACE4-1', 'SPACE5-1']

In [50]:
util.list_available_variables()

,Kind,VariableName,KeyValue,Units
0,Outputvariable,Lights Electricity Energy,SPACE1-1 LIGHTS 1,J
1,Outputvariable,Lights Electricity Energy,SPACE2-1 LIGHTS 1,J
2,Outputvariable,Lights Electricity Energy,SPACE3-1 LIGHTS 1,J
3,Outputvariable,Lights Electricity Energy,SPACE4-1 LIGHTS 1,J
4,Outputvariable,Lights Electricity Energy,SPACE5-1 LIGHTS 1,J
5,Outputvariable,Electric Equipment Electricity Energy,SPACE1-1 ELECEQ 1,J
6,Outputvariable,Electric Equipment Electricity Energy,SPACE2-1 ELECEQ 1,J
7,Outputvariable,Electric Equipment Electricity Energy,SPACE3-1 ELECEQ 1,J
8,Outputvariable,Electric Equipment Electricity Energy,SPACE4-1 ELECEQ 1,J
9,Outputvariable,Electric Equipment Electricity Energy,SPACE5-1 ELECEQ 1,J


In [48]:
util.list_available_actuators()


,Kind,ComponentType,ControlType,ActuatorKey,Units
0,Actuator,Schedule:Compact,Schedule Value,OCCUPY-1,[ ]
1,Actuator,Schedule:Compact,Schedule Value,LIGHTS-1,[ ]
2,Actuator,Schedule:Compact,Schedule Value,EQUIP-1,[ ]
3,Actuator,Schedule:Compact,Schedule Value,INFIL-SCH,[ ]
4,Actuator,Schedule:Compact,Schedule Value,ACTSCHD,[ ]
...,...,...,...,...,...
1209,Actuator,Outdoor Air System Node,Drybulb Temperature,CENTRAL CHILLER CONDENSER INLET NODE,[C]
1210,Actuator,Outdoor Air System Node,Wetbulb Temperature,CENTRAL CHILLER CONDENSER INLET NODE,[C]
1211,Actuator,Outdoor Air System Node,Wind Speed,CENTRAL CHILLER CONDENSER INLET NODE,[m/s]
1212,Actuator,Outdoor Air System Node,Wind Direction,CENTRAL CHILLER CONDENSER INLET NODE,[degree]


In [49]:
util.list_available_meters()

,Kind,MeterName,Units
0,Outputmeter,Electricity:Facility,J
1,Outputmeter,Electricity:Building,J
2,Outputmeter,Electricity:Zone:SPACE1-1,J
3,Outputmeter,Electricity:SpaceType:GENERAL,J
4,Outputmeter,InteriorLights:Electricity,J
...,...,...,...
113,Outputmeter,Carbon Equivalent:Facility,kg
114,Outputmeter,CarbonEquivalentEmissions:Carbon Equivalent,kg
115,Outputmeter,MYGENERALLIGHTS,J
116,Outputmeter,MYBUILDINGELECTRIC,J
